# 대구시 구 별 CCTV구역과 ECLO 비교

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 한글 문제
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...  sorry~~~')

# train데이터 가져오기

In [3]:
# 위도, 경도 반환하는 함수
from tqdm import tqdm
train_df_copy = train_addr.head(10)
def geocoding(address):
    try:
        geo_local = Nominatim(user_agent='South Korea')  #지역설정
        location = geo_local.geocode(address)
        geo = [location.latitude, location.longitude]
        return geo
    except:
        return [0,0]
# 실행
for idx,addr in enumerate(tqdm(train_df_copy.시군구)):
    train_df_copy.loc[idx,'latitude'] = geocoding(addr)[0]
    train_df_copy.loc[idx,'longitude'] = geocoding(addr)[1]
train_df_copy.head()

NameError: name 'train_addr' is not defined

In [ ]:
# 위도, 경도 반환하는 함수
from tqdm import tqdm
from geopy.geocoders import Nominatim

train_df_copy = train_addr.head(10)
def geocoding(address):
    try:
        geo_local = Nominatim(user_agent='South Korea')  #지역설정
        location = geo_local.geocode(address)
        geo = [location.latitude, location.longitude]
        return geo
    except:
        return [0,0]
# 실행
for idx,addr in enumerate(tqdm(train_df_copy.시군구)):
    train_df_copy.loc[idx,'latitude'] = geocoding(addr)[0]
    train_df_copy.loc[idx,'longitude'] = geocoding(addr)[1]
train_df_copy.head()

In [ ]:
train_df = pd.read_csv('./open/train.csv', encoding="UTF-8", low_memory=False)
train_addr = train_df[['ID', '시군구', '사고유형', '법규위반', 'ECLO']]

# 추가로 동별 eclo 평균 구하기
average_eclo_by_dong = train_addr.groupby('시군구')['ECLO'].mean()

# "동리로"에 대한 ECLO 평균값을 가지고 있는 데이터프레임과 기존의 train 데이터프레임을 병합
train_with_avg_eclo = pd.merge(train_addr, average_eclo_by_dong, how='left', left_on='시군구', right_index=True, suffixes=('', '_평균값'))


# CCTV설치대수 구하기

In [ ]:

# CSV 파일 읽기
cctv_df = pd.read_csv('./open/external_open/대구 CCTV 정보.csv', encoding="cp949", low_memory=False)
cctv_df['시군구명'] = cctv_df['시군구명'].replace({'군위군': '군위', '달성': '달성군'})
# "단속구분"이 01인 행만 가져오기 단속구분은 int형임. 01->속도단속카메라
filtered_cctv_df = cctv_df[cctv_df['단속구분'] == 1]

# "시군구명"에 대한 CCTV 개수 계산
cctv_count_by_sigungu = filtered_cctv_df['시군구명'].value_counts()

# 데이터 확인
print(cctv_count_by_sigungu)

# 히스토그램 표현
plt.figure(figsize=(12, 6))
cctv_count_by_sigungu.plot(kind='bar', color='skyblue')
plt.title('CCTV Count Distribution by Sigungu (단속구분: 01)')
plt.xlabel('시군구명')
plt.ylabel('CCTV 개수')
plt.xticks(rotation=45, ha='right')
plt.show()

<!-- # 시군구별 CCTV분포도 -->

# 시군구별 ECLO분포도

In [ ]:
eclo_sum_by_sigungu = train_data.groupby('시군구')['ECLO'].sum().reset_index()

# 내림차순으로 정렬
eclo_sum_by_sigungu = eclo_sum_by_sigungu.sort_values(by='ECLO', ascending=False)

# 분포도 표시
plt.figure(figsize=(12, 6))
plt.bar(eclo_sum_by_sigungu['시군구'], eclo_sum_by_sigungu['ECLO'])
plt.xlabel('시군구')
plt.ylabel('ECLO 값의 합')
plt.title('시군구별 ECLO 값의 합 분포도')
plt.xticks(rotation=45)
plt.show()

# 시군구별 속도제한 CCTV 갯수 데이터프레임

In [ ]:
cctv_count_by_sigungu = cctv_df.groupby('시군구명').size().reset_index(name='CCTV 개수')

# 결과 출력
print(cctv_count_by_sigungu)

# 시군구별 ECLO합 데이터 프레임

In [ ]:
eclo_count_by_sigungu = train_data.groupby('시군구').size().reset_index(name='eclo합')

# 결과 출력
print(eclo_count_by_sigungu)

In [ ]:
# train 데이터프레임에서 "시군구"를 "시군구명"으로 변경
train_data.rename(columns={'시군구': '시군구명'}, inplace=True)

# 두 데이터프레임을 "시군구명"을 기준으로 합치기
merged_data = pd.merge(train_data, cctv_df, on='시군구명')

# 시군구별 ECLO 값의 합 구하기
eclo_sum_by_sigungu = train_data['시군구명'].value_counts()

# cctv_count_by_sigungu 구하기
cctv_count_by_sigungu = cctv_df['시군구명'].value_counts() * 50

# 막대 그래프 표시
plt.figure(figsize=(15, 6))



# CCTV 개수 막대 그래프
plt.bar(cctv_count_by_sigungu.index, cctv_count_by_sigungu.values, 0.25, color='skyblue', label='CCTV 개수', align='edge')

# ECLO 값의 합 막대 그래프
plt.bar(eclo_sum_by_sigungu.index, eclo_sum_by_sigungu.values, 0.25, color='orange', alpha=1, label='ECLO 합계')

# 축과 레이블 설정
plt.xlabel('시군구명')
plt.ylabel('개수')
plt.title('CCTV 개수 및 시군구별 ECLO 값의 합 분포도')
plt.xticks(index + bar_width / 2, cctv_count_by_sigungu.index, rotation=45, ha='right')

# 범례 표시
plt.legend()

# 그래프 표시
plt.show()